In [1]:
import andi
import numpy as np
from tensorflow.keras.models import load_model
from utils import data_norm, data_reshape, many_net_uhd, my_atan

Generating trajectories to test the data. 3 trajectories for each dimension of inference and classification tasks. This requires the AnDi package, downloadable at https://github.com/AnDiChallenge

In [3]:
AD = andi.andi_datasets()
traj_length = 200

X1, Y1, X2, Y2, X3, Y3 = AD.andi_dataset(N = 10, tasks = [1,2,3], dimensions = [1,2,3],
                                        min_T = traj_length, max_T = traj_length+1, )

Creating a dataset for task(s) [1, 2, 3] and dimension(s) [1, 2, 3].
Generating dataset for dimension 1.
Generating dataset for dimension 2.
Generating dataset for dimension 3.


# Inference of 1d trajectories

The output of the inference networks is the inferred anomalous exponent for each trajectory. Trajectories must be of equal length

Importing the networks used for inference in 1d

In [4]:
centers_inf_1d = [25, 50, 65, 75, 125, 165, 225, 
                  325, 425, 525, 625, 725, 825, 925]      # Length that the neural networks are trained for
meta_model_inf_1d = []
for i in centers_inf_1d: 
    m = load_model('nets/inference_nets/1d/inference_1D_'+str(i)+'.h5')
    
    meta_model_inf_1d.append(m)                          # A list of neural networks that are trained for specific length
    

# Network trained on trajectories of 165 is used to analyze trajectories of length 200

In [5]:
#choosing the net
net = meta_model_inf_1d[5]                              
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X1[0],dim=1,task=1)

#reshaping the data
data_rs = data_reshape(data,bs=bs,dim=1)

#prediction on trajectories of length 200 using a net trained on traj of length 165
pred_200_u165 = net.predict(data_rs)
print('predicted exponents', pred_200_u165.flatten())
print('ground truth', Y1[0])

predicted exponents [0.3297272  1.4824448  1.5020162  0.93287134 1.6899136  1.5477296
 1.8939477  1.5448136  1.222714   0.18381149]
ground truth [0.3, 1.45, 2.0, 1.25, 1.35, 1.3, 1.8, 1.55, 0.65, 0.2]


# Network trained on trajectories of 225 is used to analyze trajectories of length 200


In [6]:
#choosing the net
net = meta_model_inf_1d[6]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X1[0],dim=1,task=1)

#reshaping the data
data_rs = data_reshape(data,bs=bs,dim=1)

#prediction on trajectories of length 200 using a net trained on traj of length 225
pred_200_u225 = net.predict(data_rs)
print('predicted exponents', pred_200_u225.flatten())
print('ground truth', Y1[0])

predicted exponents [0.4293936  1.4391744  1.6014469  0.90228367 1.6145022  1.4664931
 1.8764971  1.6154985  1.3431249  0.1748659 ]
ground truth [0.3, 1.45, 2.0, 1.25, 1.35, 1.3, 1.8, 1.55, 0.65, 0.2]


# Using the combination of nearest nets
This demonstrates how using the two nearest networks (which in this case is 165 and 225) help to analyze the exponents more accurately. 

In [8]:
pred_200_comb = many_net_uhd(nets = meta_model_inf_1d, traj_set = X1[0], centers = centers_inf_1d ,dim = 1, task =1)
print('predicted exponents',pred_200_comb)
print('ground truth', Y1[0])

predicted exponents [0.3878659  1.4572036  1.5600173  0.9150285  1.6459236  1.5003417
 1.8837682  1.5860465  1.2929536  0.17859322]
ground truth [0.3, 1.45, 2.0, 1.25, 1.35, 1.3, 1.8, 1.55, 0.65, 0.2]


# Classification in 2d

The output of the classiciation networks is an array giving the probability that the trajectory belongs to a model class. The classes, as in AnDi are [attm,ctrw,fbm,lw,sbm]

In [9]:
centers_class_2d = [25, 65, 125, 225, 425]
meta_model_class_2d = []
for i in centers_class_2d: 
    m = load_model('nets/classification_nets/2d/classification_2D_'+str(i)+'.h5')
    
    meta_model_class_2d.append(m)
    

# Network trained on trajectories of 165 is used to analyze trajectories of length 200

In [12]:
#choosing the net
net = meta_model_class_2d[2]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X2[1],dim=2,task=2)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=2)
#prediction on trajectories of length 200 using a net trained on traj of length 165
cla_200_u125 = net.predict(data_rs)
print("probability of each model class",'\n',cla_200_u125)
print("predicted most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cla_200_u125[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y2[1][i])])

probability of each model class 
 [[7.55120896e-11 9.55009682e-12 8.45531556e-09 1.00000000e+00
  7.35940753e-11]
 [4.12690043e-01 1.97531102e-04 3.54377985e-01 4.17918200e-04
  2.32316568e-01]
 [4.75204699e-02 9.51892078e-01 6.10700567e-07 5.86562208e-04
  3.66807512e-07]
 [2.75578916e-01 5.69130898e-05 4.53345060e-01 1.06699485e-03
  2.69952208e-01]
 [5.23142051e-04 8.55310427e-05 9.98525202e-01 2.45737169e-07
  8.65959795e-04]
 [2.16318071e-02 9.93439426e-06 1.26819694e-02 7.47587364e-06
  9.65668857e-01]
 [1.19896204e-10 2.22436826e-11 3.15713677e-08 1.00000000e+00
  2.17166406e-11]
 [9.98183548e-01 1.60190160e-03 5.55786528e-06 1.94007720e-04
  1.50409105e-05]
 [4.80145589e-02 3.98567354e-04 8.80312324e-02 3.31699921e-05
  8.63522470e-01]
 [3.02954833e-03 9.96969521e-01 6.27174686e-08 9.18807757e-07
  3.77421685e-08]]
predicted most likely model
lw
attm
ctrw
fbm
fbm
sbm
lw
attm
sbm
ctrw
Ground truth
lw
attm
ctrw
fbm
fbm
sbm
lw
attm
sbm
ctrw


# Combining the nets

In [13]:
cla_200_comb = many_net_uhd(nets = meta_model_class_2d, traj_set = X2[1], centers = centers_class_2d ,dim = 2, task =2)
cla_200_comb = cla_200_comb.reshape(-1,5)
print("probability of each model class",'\n',cla_200_comb)
print("most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cla_200_comb[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y2[1][i])])    

probability of each model class 
 [[4.19571461e-11 1.55693045e-11 1.85102422e-08 1.00000000e+00
  2.61412159e-11]
 [3.59516710e-01 5.59711247e-04 4.19125468e-01 1.70295214e-04
  2.20627859e-01]
 [3.69644985e-02 9.62798476e-01 5.25411906e-06 2.29561221e-04
  2.13348744e-06]
 [2.35607415e-01 3.65132262e-04 4.98450071e-01 1.96813699e-03
  2.63609260e-01]
 [2.55438033e-04 1.34457630e-04 9.99267757e-01 1.10629216e-07
  3.42214567e-04]
 [6.39395416e-03 5.90570380e-06 3.69449146e-03 2.11602037e-06
  9.89903569e-01]
 [3.04736722e-11 6.45942111e-12 8.81077078e-09 1.00000000e+00
  5.54974296e-12]
 [9.99222279e-01 6.74419338e-04 1.21888761e-05 7.31877299e-05
  1.80117531e-05]
 [1.99536663e-02 2.01059884e-04 3.78071629e-02 1.58722614e-05
  9.42022324e-01]
 [2.73457030e-03 9.97263551e-01 3.58038989e-07 1.32219566e-06
  1.48754381e-07]]
most likely model
lw
fbm
ctrw
fbm
fbm
sbm
lw
attm
sbm
ctrw
Ground truth
lw
attm
ctrw
fbm
fbm
sbm
lw
attm
sbm
ctrw


# Segmentation in 3d

Network to infer the exponents of the two segments and the switching point

In [15]:
model_a_t3d = load_model('nets/segmentation_nets/3d/T33D_inf.h5')

Network to classify the model of the second segment

In [17]:
class2_3d = load_model('nets/segmentation_nets/3d/T33D_c2.h5')

The output is [a1,a2,sin(2pi*t/T),cos(2pi*t/T)]

In [18]:
#finding out the block size used by the chosen net
bs = model_a_t3d.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X3[2],dim=3,task=3)

#reshaping the data
data_rs = data_reshape(data,bs=bs,dim=3) 

#prediction on trajectories of length 200 using a net trained on traj of length 165
inf_seg = model_a_t3d.predict(data_rs)
pred_t = my_atan(inf_seg[:,2],inf_seg[:,3])*200/(2*np.pi)
cl2_3d = class2_3d.predict(data_rs)

print('predicted first exponents', inf_seg[:,0])
print('ground truth first exponents', Y3[2][:,3])
print('predicted second exponents', inf_seg[:,1])
print('ground truth second exponents', Y3[2][:,5])
print('predicted time and exponents', pred_t)
print('ground truth time', Y3[2][:,1])

predicted first exponents [0.5797447  0.6147374  0.3347436  1.230671   0.5097573  1.0161972
 0.42441815 0.455482   0.5678468  0.72195107]
ground truth first exponents [1.6  0.1  0.15 1.6  1.1  0.65 0.9  0.6  1.4  1.2 ]
predicted second exponents [1.829998   1.5415918  0.9014102  1.0125735  0.3182495  1.7972714
 1.8258257  0.40838283 1.124426   0.45345384]
ground truth second exponents [1.9  1.65 1.15 0.55 0.15 1.85 1.75 0.1  0.7  0.7 ]
predicted time and exponents [ 27.36010047  35.14043156 189.22398873 117.51175227  47.88340925
   7.48563974  32.20398904 155.44904384 191.9498973  176.26879801]
ground truth time [ 36.  35. 192. 120.  50.  13.  32. 156. 179. 177.]


In [19]:
print("probability of each model class for second segment",'\n',cl2_3d)
print("most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cl2_3d[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y3[2][i,2])])    

probability of each model class for second segment 
 [[7.1035953e-05 6.7777168e-05 9.9347335e-01 4.8248051e-03 1.5630369e-03]
 [5.2410032e-04 1.8390221e-05 8.5001326e-01 8.8109989e-03 1.4063327e-01]
 [2.8311330e-01 1.2330370e-01 3.5173729e-01 3.2255903e-02 2.0958991e-01]
 [7.9043537e-02 1.0802548e-04 1.7758565e-01 5.3537340e-04 7.4272734e-01]
 [3.6448956e-02 3.2949399e-03 9.5901537e-01 2.5676782e-05 1.2150351e-03]
 [3.0859115e-03 4.3872508e-04 1.2599690e-01 1.8379205e-03 8.6864060e-01]
 [1.7793437e-07 7.3853874e-07 2.5342579e-06 9.9999619e-01 3.4595112e-07]
 [2.5060678e-01 7.0500278e-01 4.1326419e-02 2.1839018e-04 2.8455905e-03]
 [2.6846834e-04 7.1689494e-05 2.1907852e-03 9.9591404e-01 1.5549882e-03]
 [2.6703227e-01 6.7155123e-01 3.2111373e-02 3.0504141e-03 2.6254687e-02]]
most likely model
fbm
fbm
fbm
sbm
fbm
sbm
lw
ctrw
lw
ctrw
Ground truth
lw
fbm
fbm
sbm
sbm
fbm
attm
fbm
lw
fbm
